In [8]:
def readfile(file):
    with open(file) as f:
        contents = f.read()[:-1].split('\n\n')
    
    mapping_dictionary = {}

    seed_data = contents[0].split(": ")
    map_data = contents[1:]
    mapping_dictionary[seed_data[0]] = [int(i) for i in seed_data[1].split()]
    
    map_content_dictionary = {}
    for i in map_data:
        map_content_data = i.split(" map:\n")
        map_content_list = []
        for map in map_content_data[1].split("\n"):
            mapping_rule = {0:'destination_range_start',1:'source_range_start',2:'range_length'}
            mapping_content = {}
            for j in mapping_rule.keys():
                mapping_content[mapping_rule[j]] = int(map.split()[j])
            map_content_list.append(mapping_content)
        map_content_dictionary[map_content_data[0]] = map_content_list
    
    mapping_dictionary['mapping'] = map_content_dictionary

    return mapping_dictionary

def seed_mapper(identified_value, source, mapping):
    current_map_key = [i for i in mapping.keys() if i.split('-to-')[0]==source][0]
    active_map = mapping[current_map_key]

    new_identified_value = identified_value

    for map in active_map:
        if identified_value>=map['source_range_start'] and identified_value<map['source_range_start']+map['range_length']:
            new_identified_value = map['destination_range_start']+identified_value-map['source_range_start']
            break

    next_destination = current_map_key.split('-to-')[1]

    return new_identified_value if next_destination=='location' else seed_mapper(new_identified_value, next_destination, mapping)


def aoc_5a_solution(file):
    seed_and_mapping = readfile(file)
    location_list = [seed_mapper(seed,'seed',seed_and_mapping['mapping']) for seed in seed_and_mapping['seeds']]
    return min(location_list)

aoc_5a_solution('input.txt')

157211394

In [11]:
def aoc_5b_solution(file):
    seed_and_mapping = readfile(file)
    seed_pair_list = [(seed_start, seed_and_mapping['seeds'][index+1]) for index, seed_start in enumerate(seed_and_mapping['seeds']) if index%2==0]
    location_list = []
    for seed_pair in seed_pair_list:
        for seed in range(seed_pair[0],seed_pair[0]+seed_pair[1]):
            location_list.append(seed_mapper(seed,'seed',seed_and_mapping['mapping']))
    return min(location_list)

aoc_5b_solution('input.txt')